In [1]:
import os
import sys
global chunks
import json
chunks = []
def loaddata(location = r"C:\Users\VectorStorage\Text.pdf"):
    from langchain.document_loaders import PyPDFLoader
    loader = PyPDFLoader(location)
    pages = loader.load()
    
    from langchain.text_splitter import CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=150,
        length_function=len
    )
    docs = text_splitter.split_documents(pages)
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500,
        chunk_overlap = 150
    )
    global chunks
    chunks = text_splitter.split_documents(docs)
    
    import openai
    openai.api_key = ''
    embd = []
    for chunk in chunks:
        response = openai.Embedding.create(input= chunk.page_content, model="text-embedding-ada-002")
        embd.append(response)
    if os.path.exists('embedding_index.faiss'):
        os.remove('embedding_index.faiss')
    embeddings = [emb['data'][0]['embedding'] for emb in embd]
    embeddings = np.array(embeddings, dtype=np.float32)

    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)

    index.add(embeddings)

    faiss.write_index(index, 'embedding_index.faiss')

In [2]:
import numpy as np
import faiss
import os
import sys
import openai
import json
def qna():
    if chunks is None:
        print("Error: Data is not loaded successfully. Please run 'loaddata' first.")
        return
    query = input("Question : ")
    response = openai.Embedding.create(input=query, model="text-embedding-ada-002")
    query_emb = response['data'][0]['embedding']
#     query_emb = np.array(query_emb, dtype=np.float32)
    
    if os.path.exists('embedding_index.faiss'):
        index = faiss.read_index('embedding_index.faiss')
    else:
        raise FileNotFoundError("Faiss index file not found.")
    
    query_embedding = np.array([query_emb])
    k = 5
    distances, indices = index.search(query_embedding, k)
    indices = indices[0]
    context = ''
    for indice in indices:
        context = context + ' ' + chunks[indice].page_content
        
    with open("memory.json", "r") as outfile:
        json_object = json.load(outfile)
        mem = ""
        for objects in json_object:
            mem = mem + "Quesion - " + objects["question"] + "Answer - " + objects["answer"]
            
    
    messages = [
        {
        'role': 'system',
        'content': f"Use only the following pieces of context to answer the question at the end. "
                    f"Do not make answers by yourself, just give the answers with respect to the context. "
                    f"If the question is not with respect to the context, just give 'I don't know' as output and don't make answers. \n "
                    f"If you don't know the answer, just say that you don't know. Use three sentences at maximum. "
                    f"Keep the answer as concise as possible. Always say 'thanks for asking!' at the end of the answer. "
                    f"Additionally, consider the previous 4 conversations for a follow-up question:\n\n {mem} \n"
                    f"Do not make answers by yourself, just give the answers with respect to the context. "
                    f"If the question is not with respect to the context, just give 'I don't know' as output and don't make answers. \n "
                    f"\n  Context: {context}"
        },
        {'role': 'user', 'content': f"{query}"}
    ]


    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
    )
    answer = response['choices'][0]['message']['content']
    
    print("Answer->",response['choices'][0]['message']['content'])
    try:
        with open("memory.json", "r") as infile:
            memory = json.load(infile)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        memory = []
    
    cache = {
        "question": query,
        "answer": answer
    }
    memory.append(cache)
    
    memory = memory[-4:]
    
    with open("memory.json", "w") as outfile:
        json.dump(memory, outfile)

In [3]:
loaddata()

In [ ]:
qna()

In [ ]:
#     messages2 = [
#         {'role': 'system', 'content': f"Use the text to figure out if the question is related to a previous question or not. \
#          Just check if the question given by the user is relevant to the previous questions or not, and then after this give \
#          the response as a full regenerated question which is self sufficient to understand and can be the full question. \
#          directly. If the new question is not related to the previous questions then just pass the \
#          question as it is. \n {mem}"},
#         {'role': 'user', 'content': f"{query}"}
#     ]
    
#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=messages2,
#         temperature=0,
#     )
#     queryf = response['choices'][0]['message']['content']